In [15]:
import requests
import time
import pandas as pd
import numpy as np

# Data extraction

In [41]:
token = '7890956e-a2bf-42ef-863e-83cdaae50174'
headers={'accept':'application/json', 'X-API-KEY': token}
films_path = "/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/andru_preprocessing/film_num_list.txt"
lst_success = 2728


In [34]:
films = []

In [35]:
class Film:
    def __init__(self, id, name, rewiews):
        self.id = id
        self.name = name
        self.rewiews = rewiews

In [36]:
def get_film(id):
    response = requests.get('https://kinopoiskapiunofficial.tech/api/v2.2/films/{0}'.format(id), headers=headers)
    filmJson = response.json()
    if 'error' in filmJson:
      print(filmJson)
      print(int(1) / int(0))
    if 'nameRu' not in filmJson:
      return None
    name = filmJson['nameRu']
    # print(filmJson)
    reviews = []
    reviewsCount = filmJson['reviewsCount']
    count = reviewsCount / 20 if reviewsCount % 20 == 0 else reviewsCount / 20 + 1
    for i in range(1, int(count) + 1):
        response = requests.get('https://kinopoiskapiunofficial.tech/api/v1/reviews?filmId={0}&page={1}'.format(id, i), headers=headers)
        j = response.json()
        for r in j['reviews']:
            reviews.append(r)
        time.sleep(0.005)
    return Film(id, name, reviews)

In [39]:
df = pd.read_csv(films_path, header=None)

In [42]:
f = get_film(1793)

In [ ]:
f.rewiews

# necessary column names
reviewDescription
reviewType
userNegativeRating
userPositiveRating

In [43]:
ids = list(df[0])
ids.sort()
for id in filter(lambda x: x > lst_success, ids):
    print('\r' + str(id), sep=' ', end='', flush=True)
    f = get_film(id)
    if f is not None:
      films.append(f)
    

2728{'timestamp': '2022-04-09T14:17:07.928+00:00', 'status': 402, 'error': 'Payment Required', 'path': '/api/v2.2/films/2728'}


ZeroDivisionError: ignored

In [44]:
# Save data
films_copy = films.copy()

In [45]:
import pandas as pd
import numpy as np

# Create empty DataFrame
columns = ['film_id', 'film_name', 'film_review', 'pos/neg', 'user_likes', 'user_dislikes']
reviews_for_csv = pd.DataFrame(columns = columns)
reviews_for_csv

,film_id,film_name,film_review,pos/neg,user_likes,user_dislikes


In [46]:
# Add reviews to dataframe
for film in films:
  film_descr = {}
  film_descr['film_name'] = film.name
  film_descr['film_id'] = int(film.id)
  for review in film.rewiews:
    film_descr['film_review'] = review['reviewDescription']
    film_descr['pos/neg'] = review['reviewType']
    film_descr['user_likes'] = int(review['userPositiveRating'])
    film_descr['user_dislikes'] = int(review['userNegativeRating'])
    reviews_for_csv = reviews_for_csv.append(film_descr, ignore_index = True)

reviews_for_csv.head()

,film_id,film_name,film_review,pos/neg,user_likes,user_dislikes
0,1794,50 первых поцелуев,Мелодрама? Хорошая мелодрама? Хорошая мелодрам...,POSITIVE,1,1
1,1794,50 первых поцелуев,Генри влюбляется в девушку из кафе Люси. Люси ...,POSITIVE,1,0
2,1794,50 первых поцелуев,Как волнительно первое свидание и сладок первы...,POSITIVE,1,0
3,1794,50 первых поцелуев,"Очень милый фильм, наполненный морем, солнцем ...",POSITIVE,0,0
4,1794,50 первых поцелуев,Добротная лирическо-любовная комедия с элемент...,POSITIVE,7,0


In [ ]:
# Save DataFrame with reviews to .csv
reviews_for_csv.to_csv('films_reviews.csv')
reviews_for_csv.shape

(6899, 6)

In [ ]:
# Copy .csv file 
%cp -av '/content/films_reviews.csv' '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3'

'/content/films_reviews.csv' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/films_reviews.csv'


In [ ]:
df_test = pd.DataFrame(columns = ['num_uniq'])
df_test['num_uniq'] = reviews_for_csv['film_id'].value_counts()

,num_uniq
335,602
448,500
435,469
325,402
382,368


In [ ]:
df_test[df_test['num_uniq'] > 10].shape

(80, 1)

In [ ]:
# соотношение Good / Bad > 0.9
# df_test[df_test['user_likes'] / df_test['user_dislikes'] > 0.9]

reviews_for_csv['likes_ratio'] = 0

def find_relevant(x):
  print(x)
  if x['user_dislikes'] != 0: 
    x['likes_ratio'] = x['user_likes'] / x['user_dislikes']
  return x

test = reviews_for_csv.apply(lambda x: find_relevant(x))
test.head()

0        306
1        306
2        306
3        306
4        306
        ... 
6894    1786
6895    1786
6896    1789
6897    1789
6898    1789
Name: film_id, Length: 6899, dtype: object


KeyError: ignored

In [ ]:
reviews_for_csv.head()

,film_id,film_name,film_review,pos/neg,user_likes,user_dislikes,likes_ratio
0,306,Рекрут,К концу 1970-х годов новозеландский кинематогр...,POSITIVE,0,0,0
1,306,Рекрут,Проходясь по фильмографии моего любимого Колин...,NEGATIVE,2,2,0
2,306,Рекрут,Шпионский триллер Роджера Дональдсона «Рекрут»...,POSITIVE,3,0,0
3,306,Рекрут,Фильмы начала 2000-годов всегда привлекали мое...,POSITIVE,2,1,0
4,306,Рекрут,К студенту Джеймсу Клейтону обращается агент Ц...,POSITIVE,3,0,0


# Filter film names:

In [ ]:
# найти максимум среди id
# собрать новые рецензии в таблицу
# объединить с csv файлом

# оставить те ревью, у которых больше 1 лайка
# поделить лайки на дизлайки и оставить фильмы, у которых соотношение больше 0.8 (с учетом деления на ноль, тогда оставить 1)
# оставить фильмы, у которых больше 3 ревью

### Raw data preprocessing

In [ ]:
old_df = pd.read_csv('/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/films_reviews.csv', index_col=0)

In [65]:
df = pd.concat([old_df, reviews_for_csv], ignore_index = True)
df.shape

(10702, 6)

In [82]:
a = df.copy()
a = a[a['user_likes'] > 3]
display(a.shape)
display(len(list(set(a['film_id']))))

(9302, 6)

357

In [88]:
b = a.copy()
b['like_ratio'] = 0
b['user_dislikes'] = b['user_dislikes'].replace(0, -1)
b['like_ratio'] = b['user_likes'] / b['user_dislikes']
print(b.shape)
b = b[b['like_ratio'] > 0.8]
# b = b[b['like_ratio'] > 0.8]
print(b.shape)
b.head(20)

(9302, 7)
(7418, 7)


,film_id,film_name,film_review,pos/neg,user_likes,user_dislikes,like_ratio
6,306,Рекрут,"С фильмом «Рекрут» (The Recruit, 2003) получае...",NEUTRAL,4,4,1.0
9,306,Рекрут,"Джеймс Клэйтон — талантливый программист, выну...",POSITIVE,4,1,4.0
14,306,Рекрут,Рекрут представляет собой изощренный и закруче...,NEUTRAL,14,5,2.8
15,306,Рекрут,"Фильм довольно интересный. Нет, правда. Он, ко...",NEUTRAL,5,3,1.666667
16,306,Рекрут,"Фильм, вышедший в 2002-ом году и собравший неп...",NEGATIVE,16,6,2.666667
19,306,Рекрут,"Всё не то, чем кажется.\r\n\r\nЭто один из луч...",POSITIVE,9,5,1.8
20,306,Рекрут,Я определённо люблю фильмы с Колином Фарреллом...,POSITIVE,9,3,3.0
21,306,Рекрут,Один из лучших триллеров на тему американских ...,POSITIVE,10,4,2.5
22,306,Рекрут,"Что вы знаете о ЦРУ? То, что пишут в газетах и...",POSITIVE,10,2,5.0
25,306,Рекрут,"Шпионский триллер должен быть продуманным, неп...",NEGATIVE,70,16,4.375


In [95]:
# display(b.shape)
# display(b[b['film_name'].value_counts() > 5].shape)
# b['film_id'] > 5

count_reviews = b['film_name'].value_counts().to_frame()
print(count_reviews.shape)
count_reviews = count_reviews[count_reviews['film_name'] >= 5]
print(count_reviews.shape)

idxs = count_reviews.index

(335, 1)
(176, 1)


In [96]:
idxs

Index(['Помни', 'Форрест Гамп', 'Зеленая миля', 'Крестный отец', 'Король Лев',
       'Американская история X', 'Криминальное чтиво', 'Гладиатор',
       'Искусственный разум', 'Дракула',
       ...
       'Молодая кровь', 'Арарат', 'К2: Предельная высота', 'Сейчас и тогда',
       'Защищая твою жизнь', 'Мистер Сталь', 'Побег невозможен',
       'Милашка в розовом', 'Призрак и миссис Мьюр', 'Это старое чувство'],
      dtype='object', length=176)

In [97]:
list_idxs = idxs.to_list()
list_idxs

['Помни',
 'Форрест Гамп',
 'Зеленая миля',
 'Крестный отец',
 'Король Лев',
 'Американская история X',
 'Криминальное чтиво',
 'Гладиатор',
 'Искусственный разум',
 'Дракула',
 'Анастасия',
 'Гражданин Кейн',
 'Мертвец',
 'Привидение',
 'Мосты округа Мэдисон',
 'Большой Лебовски',
 'Грязные танцы',
 'Звонок',
 'Амадей',
 'Казино',
 '12 обезьян',
 'Скала',
 'Магнолия',
 'Чужой 3',
 '50 первых поцелуев',
 'Огни большого города',
 'Битлджюс',
 'Фарго',
 'Кейт и Лео',
 'Простая история',
 'Звёздные войны: Эпизод 6 — Возвращение Джедая',
 'Рекрут',
 'Смерть ей к лицу',
 'Вспомнить всё',
 'Убить пересмешника',
 'Аладдин',
 'История рыцаря',
 'Чего хотят женщины',
 'Королевская битва',
 'Мумия возвращается',
 'Трудная мишень',
 'Тельма и Луиза',
 'Послание в бутылке',
 'Бездна',
 'Лжец, лжец',
 'Братство волка',
 'Знакомство с родителями',
 'Быстрый и мертвый',
 'Геркулес',
 'Горбун из Нотр Дама',
 'Ганди',
 'Энни Холл',
 'Суперзвезда',
 'Крик 3',
 'Мэй',
 'Несколько хороших парней',
 'Оборо

In [100]:
filtered_films = pd.DataFrame(list_idxs, columns = ['filtered_film_names'])
filtered_films.head()
# list_idxs

,filtered_film_names
0,Помни
1,Форрест Гамп
2,Зеленая миля
3,Крестный отец
4,Король Лев


In [101]:
df.head()

,film_id,film_name,film_review,pos/neg,user_likes,user_dislikes
0,306,Рекрут,К концу 1970-х годов новозеландский кинематогр...,POSITIVE,0,0
1,306,Рекрут,Проходясь по фильмографии моего любимого Колин...,NEGATIVE,2,2
2,306,Рекрут,Шпионский триллер Роджера Дональдсона «Рекрут»...,POSITIVE,3,0
3,306,Рекрут,Фильмы начала 2000-годов всегда привлекали мое...,POSITIVE,2,1
4,306,Рекрут,К студенту Джеймсу Клейтону обращается агент Ц...,POSITIVE,3,0


In [102]:
# Save DataFrame with reviews to .csv
df.to_csv('films_reviews_1.csv')
filtered_films.to_csv('filtered_films.csv')
reviews_for_csv.shape

(3803, 6)

In [103]:
# Copy .csv file 
%cp -av '/content/films_reviews_1.csv' '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3'
%cp -av '/content/filtered_films.csv' '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3'

'/content/films_reviews_1.csv' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/films_reviews_1.csv'
'/content/filtered_films.csv' -> '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/filtered_films.csv'


In [98]:
b.shape

(7418, 7)

In [ ]:
b[b['']]

In [ ]:
a['like_ratio'] = a['user_likes'] / a['user_dislikes']
a.head()

In [ ]:
a['like_ratio'] = 0
a.head()

In [77]:
def do_smth(x):
  print(x)
  
  # if x['user_dislikes'] == 0:
  #   return 1
  return 0

In [80]:
b = a.copy()

# # b = b.apply(lambda x: do_smth(x))
# b.head()

,film_id,film_name,film_review,pos/neg,user_likes,user_dislikes,like_ratio
6,306,Рекрут,"С фильмом «Рекрут» (The Recruit, 2003) получае...",NEUTRAL,4,4,0
9,306,Рекрут,"Джеймс Клэйтон — талантливый программист, выну...",POSITIVE,4,1,0
14,306,Рекрут,Рекрут представляет собой изощренный и закруче...,NEUTRAL,14,5,0
15,306,Рекрут,"Фильм довольно интересный. Нет, правда. Он, ко...",NEUTRAL,5,3,0
16,306,Рекрут,"Фильм, вышедший в 2002-ом году и собравший неп...",NEGATIVE,16,6,0


In [ ]:
df = df[df['user_likes'] > 3]


In [ ]:
red

In [55]:
old_df = pd.read_csv('/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/films_reviews.csv', index_col=0)
# new_df = pd.read_csv('/')



In [59]:
display(len(old_df)) #, reviews_for_csv
display(len(reviews_for_csv))

6899

3803

In [ ]:
#### Random choice from comment

In [57]:
reviews_for_csv.head()

,film_id,film_name,film_review,pos/neg,user_likes,user_dislikes
0,1794,50 первых поцелуев,Мелодрама? Хорошая мелодрама? Хорошая мелодрам...,POSITIVE,1,1
1,1794,50 первых поцелуев,Генри влюбляется в девушку из кафе Люси. Люси ...,POSITIVE,1,0
2,1794,50 первых поцелуев,Как волнительно первое свидание и сладок первы...,POSITIVE,1,0
3,1794,50 первых поцелуев,"Очень милый фильм, наполненный морем, солнцем ...",POSITIVE,0,0
4,1794,50 первых поцелуев,Добротная лирическо-любовная комедия с элемент...,POSITIVE,7,0


In [64]:
# import random
# ith = random.randint(0, reviews_for_csv.shape[0])
# reviews_for_csv['film_review'][ith]

'Уж больно сложносочиненная вещь. Как справедливо замечается много где, даже жанр определить непросто. Отсюда следует простейший вывод: это постмодернизм. Впрочем в отличие от Тарантино, Джармуша интересует не столько смешение форм в основном из 70-х, сколько пересечение культур и люди, которые живут на границе между этими культурами. «Мертвец» — это путешествие из культуры Востока США в культуру Дикого Запада, из культуры Дикого Запада в культуру индейцев. А дальнейшее путешествие и хождение по границам уже не фиксируется столь определенно. В общем чем дальше в лес, тем больше дров и тем интереснее. Оттого-то все происходящее на экране напоминает притчу. Но по правде сказать, не складывается впечатление, что все это рассказывается серьезно. Эдакая неестественно черная комедия. Единственное, что в фильме вполне определенно — это конец.\r\n\r\nА вот что в фильме безусловно, так это изобразительный ряд — при минимуме средств добиться столь вожделенной картины надо было постараться. Эффек

### Preparing data for training

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Подтянуть табличку с рецензиями на фильм
# Отфилтровать рецензии
# Оставить в наборе 2 колонки: название - рецензия
# Очистить рецензии
# Привести к виду, пригодному для обучения

In [3]:
path = '/content/drive/MyDrive/Учеба/2 сем/NLP/HWs/HW3/films_reviews_1.csv'
df = pd.read_csv(path, index_col=0)
display(df.head())
display(df.tail())

,film_id,film_name,film_review,pos/neg,user_likes,user_dislikes
0,306,Рекрут,К концу 1970-х годов новозеландский кинематогр...,POSITIVE,0,0
1,306,Рекрут,Проходясь по фильмографии моего любимого Колин...,NEGATIVE,2,2
2,306,Рекрут,Шпионский триллер Роджера Дональдсона «Рекрут»...,POSITIVE,3,0
3,306,Рекрут,Фильмы начала 2000-годов всегда привлекали мое...,POSITIVE,2,1
4,306,Рекрут,К студенту Джеймсу Клейтону обращается агент Ц...,POSITIVE,3,0


,film_id,film_name,film_review,pos/neg,user_likes,user_dislikes
10697,2719,Нортфорк,Братья Полиши сняли весьма нестандартное кино....,NEGATIVE,9,6
10698,2719,Нортфорк,"Это совершенно странный, малобюджетный фильм, ...",POSITIVE,10,1
10699,2719,Нортфорк,"«Northfork», этапное произведение братьев Поли...",POSITIVE,10,1
10700,2719,Нортфорк,Этот фильм — памятник детству с его сказками и...,POSITIVE,13,6
10701,2719,Нортфорк,"Вы когда-нибудь задумывались, куда же делись з...",POSITIVE,27,5


In [ ]:
df = df[df['user_likes'] > 3]
df['like_ratio'] = 0
df['user_dislikes'] = df['user_dislikes'].replace(0, -1)
df['like_ratio'] = df['user_likes'] / df['user_dislikes']
df = df[df['like_ratio'] > 0.8]

count_reviews = df['film_name'].value_counts().to_frame()
print(count_reviews.shape)
count_reviews = count_reviews[count_reviews['film_name'] >= 5]
print(count_reviews.shape)

idxs = count_reviews.index
list_idxs = idxs.to_list()

In [5]:
df = df.loc[df['film_name'].isin(list_idxs)]
df = df[['film_name','film_review']]
df = df.reset_index(drop=True)

In [6]:
import re

# Удаляю \n
# Удаляю n из 10, начиная поиск с конца
# Заменяю 2 и больше пробелов на 1

def clean_review(review):
  text = re.sub(r'\n'," ",review)
  text = re.sub(r'\r'," ",text)
  ind = text.rfind('из 10')
  if text[ind - 2] == '0': text = text[:ind - 4]
  else: text = text[:ind - 3]
  text = re.sub(r"[ ]{1,}", " ", text)
  return text

In [7]:
# Каждая строка начинается со слов "Отзыв на фильм НАЗВАНИЕ: "

def add_train_prefix(review, film_name):
  return f'Отзыв на фильм {film_name}: {review}'

In [8]:
# Добавляю необходимую <s> разметку для обучения
def train_format(review):
  return f'<s>{review}</s>'

In [9]:
def replace_name(text, name):
  return re.sub(r'Рекрут', name, text)

In [10]:
df['film_review'] = df['film_review'].apply(lambda x: clean_review(x))
df['film_review'] = df['film_review'].apply(lambda x: add_train_prefix(x, 'Рекрут'))
df['film_review'] = df['film_review'].apply(lambda x: train_format(x))
df["film_review"] = df.apply(lambda row: replace_name(row["film_review"], row["film_name"]), axis=1)

In [11]:
# Оставляем только уникальные отзывы
df = df.drop_duplicates()
df = df.reset_index(drop=True)

### Summarize main text

### Split on train / test sets

In [ ]:
# Формирую список уникальных названий фильмов
# Инициализирую train и test (copy df) датафреймы
# запускаю цикл, где прохожусь по всем названиям
# в цикле получаю имя, получаю индексы этих имен, добавляю строки в test по индексам, удаляю индексы в train


def split_on_train_test(df, name_col, train_size = 0.8):
  names = list(set(df[name_col]))
  train_df = pd.DataFrame(columns = df.columns)
  test_df = df.copy()
  # display(train_df)

  for name in names:
    idx = list(df[df['film_name'] == name].index)
    size = int(len(idx) * train_size)
    print(idx, size)
    train_idx = np.random.choice(idx, size = size, replace = False)
    train_df = pd.concat([train_df, df.loc[train_idx]], ignore_index = True)
    test_df = test_df.drop(labels = train_idx, axis = 0)
  
  return train_df, test_df


train_df, test_df = split_on_train_test(df, 'film_name', train_size = 0.8)

In [37]:
train = train_df['film_review'].to_list()
valid = test_df['film_review'].to_list()

In [38]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [39]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

In [ ]:
### 

### Summarization

In [ ]:
# Divide text on sentences and cleaning up symbol junk
def get_sentences(file_name):
  article = re.split('; |\. |\n', file_name)
  sentences = []
  
  for sentence in article:
    sentence = re.sub(r"ё", "е", sentence)
    sentence = re.sub(r"[^а-яА-Я]", " ", sentence)
    sentence = re.sub(r"[ ]{1,}", " ", sentence)
    sentence = sentence.split(" ")
    if len(sentence) < 4 and len(sentence) > 30: continue
    for wd in sentence: 
      if wd == '': sentence.remove(wd)
    sentences.append(sentence)

  for sentence in sentences:
    if len(sentence) < 4: sentences.remove(sentence)

  return sentences

In [ ]:
# Evaluate sentences similarity with cosine distance
def sentence_similarity(sent1, sent2, stopwords=None):
  if stopwords is None:
    stopwords = []

  sent1 = [w.lower() for w in sent1]
  sent2 = [w.lower() for w in sent2]

  all_words = list(set(sent1 + sent2))

  vector1 = [0] * len(all_words)
  vector2 = [0] * len(all_words)

  # build the vector for the first sentence
  for w in sent1:
    if w in stopwords:
      continue
    vector1[all_words.index(w)] += 1

  # build the vector for the second sentence
  for w in sent2:
    if w in stopwords:
        continue
    vector2[all_words.index(w)] += 1

  return 1 - cosine_distance(vector1, vector2)

In [ ]:
# Build similarity matrix between sentences
def build_similarity_matrix(sentences, stop_words):
  # Create an empty similarity matrix
  similarity_matrix = np.zeros((len(sentences), len(sentences)))
  
  for idx1 in range(len(sentences)):
    for idx2 in range(len(sentences)):
        if idx1 == idx2: #ignore if both are same sentences
            continue
        similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
  return similarity_matrix

In [ ]:
# Summary pipeline 
def generate_summary(file_name, stop_words, top_n=5):
  summarize_text = []

  # Step 1 - Read text anc split it
  sentences =  get_sentences(file_name)
  # Step 2 - Generate Similary Martix across sentences
  sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
  # Step 3 - Rank sentences in similarity martix
  sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
  scores = nx.pagerank(sentence_similarity_graph)
  max_score = sorted((scores[i] for i in range(len(scores))), reverse=True)[top_n - 1]
  print(f'Threshold score for {top_n} sentences: {max_score}')

  # Step 4 - Get extracted sentences in right sequence
  ranked_sentence = [(scores[i],s) for i,s in enumerate(sentences)]
  result = []
  for sent_set in ranked_sentence:
    if sent_set[0] >= max_score:
      result.append(' '.join(sent_set[1]))

  return result

In [ ]:
summary = generate_summary(full_text, stop_words_dict, 5)
summary

Threshold score for 5 sentences: 0.03655188631454692


['Можно вписать второго поставщика но принципиально ни у кого на рынке медизделий это картину не меняет',
 'Консерванты в России можно достать',
 'В случае все того же геля для УЗИ это может означать как просто разный срок техпроцессов например набухание геля трое суток вместо суток что сразу же означает перенастройку производства и плавающие цены на партии из за изменения себестоимости так и более серьезные вещи вроде изменения итоговой формулы',
 'Конечно в итоге мы выстроим заново работу с китайскими поставщиками понадобится другой контроль партий возможно на их стороне',
 'Хороший полимер для этого геля для УЗИ есть кроме Европы еще в Индии и в Японии']